In [1]:
model_name = 'baseline_sparse'
data_dir = './data/mlboot_dataset/'
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import scipy.sparse as sp

In [2]:
mat3 = sp.load_npz(data_dir+'mat1.npz').tolil()

In [3]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')
df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

In [4]:
mat3.tocsr()[df_train_index.tolist()].max()

96

In [9]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
y = pd.read_table(data_dir + 'mlboot_train_answers.tsv')
y.columns = ['uid','target']
df = df.merge(y, on='uid', how='left')

mat1 = sp.load_npz(data_dir+'dmat1.npz').tolil()
mat2 = sp.load_npz(data_dir+'dmat2.npz').tolil()
mat3 = sp.load_npz(data_dir+'dmat3.npz').tolil()

In [10]:
df_train_index = df[~df.target.isnull()].index
df_test_index = df[df.target.isnull()].index

X = df.loc[~df.target.isnull(),:].reset_index(drop=True)
x_te = df.loc[df.target.isnull(),:].reset_index(drop=True)

In [11]:
%%time
train_mat1 = mat1[df_train_index.tolist()]

CPU times: user 12.2 s, sys: 59.6 ms, total: 12.2 s
Wall time: 11.7 s


In [12]:
test_mat1 = mat1[df_test_index.tolist()]
train_mat2 = mat2[df_train_index.tolist()]
test_mat2 = mat2[df_test_index.tolist()]
train_mat3 = mat3[df_train_index.tolist()]
test_mat3 = mat3[df_test_index.tolist()]

In [15]:
train_cols = [u'num_keys_f1',
       u'sum_values_f1', u'num_keys_f2', u'sum_values_f2', u'num_keys_f3',
       u'sum_values_f3', u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5', u'records', u'max_days', u'min_days',
       u'sum_values_f1_std', u'num_keys_f1_std', u'sum_values_f2_std',
       u'num_keys_f2_std', u'sum_values_f3_std', u'num_keys_f3_std',
       u'sum_values_f1_max', u'num_keys_f1_max', u'sum_values_f2_max',
       u'num_keys_f2_max', u'sum_values_f3_max', u'num_keys_f3_max',
       u'sum_values_f1_mean', u'num_keys_f1_mean', u'sum_values_f2_mean',
       u'num_keys_f2_mean', u'sum_values_f3_mean', u'num_keys_f3_mean']

# Train the model
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 128,
    'max_depth' : 12,
    #'min_data' : 30,
    #'lambda_l2' : 15,
    'min_sum_hessian_in_leaf' : 0.3,
    'lambda_l1' : 2.5,
    #'max_drop' : 1,
    'learning_rate': 0.01,
    'feature_fraction': 0.7,
    'verbose': 0
}

kf = KFold(n_splits=5, shuffle=True, random_state=239)

ifold = 0

y_pred = 0
y_oof = X[['uid','target']].copy()
y_oof['target'] = np.nan

scores = []

for train_index,test_index in kf.split(X):       
    print('fold', ifold)
    
    y_tr,y_va = X.loc[train_index,'target'].values,X.loc[test_index,'target'].values
    X_tr,X_va = X.loc[train_index, train_cols].values,X.loc[test_index, train_cols].values
    X_te = x_te[train_cols]
    
    mat = test_mat1.sum(axis=0)
    ixs1 = np.asarray(mat)[0].argsort()[-5000:][::-1]
    mat = test_mat2.sum(axis=0)
    ixs2 = np.asarray(mat)[0].argsort()[-5000:][::-1]
    mat = test_mat3.sum(axis=0)
    ixs3 = np.asarray(mat)[0].argsort()[-5000:][::-1]
    
    print('prepare train')
    X_tr = sp.hstack([
        X_tr, train_mat1[:,ixs1][train_index], train_mat2[:,ixs2][train_index], train_mat3[:,ixs3][train_index]
    ]).tocsr()
    print('prepare valid')
    X_va = sp.hstack([
        X_va, train_mat1[:,ixs1][test_index], train_mat2[:,ixs2][test_index], train_mat3[:,ixs3][test_index]
    ]).tocsr()
    
    # Create the LightGBM data containers
    tr_data = lgb.Dataset(X_tr, label=y_tr) #, categorical_feature=cate_cols
    va_data = lgb.Dataset(X_va, label=y_va) #, categorical_feature=cate_cols

    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=8000,
                      early_stopping_rounds=200,
                      verbose_eval=50)
    
    yhat = model.predict(X_va, model.best_iteration)
    print(ifold,roc_auc_score(y_va,yhat))
    scores.append(roc_auc_score(y_va,yhat))
    y_oof.loc[test_index,'target'] = yhat

    print('prepare test')
    X_te = sp.hstack([
        X_te, test_mat1[:,ixs1], test_mat2[:,ixs2], test_mat3[:,ixs3]
    ]).tocsr()   
    
    ytst = model.predict(X_te, model.best_iteration)
    y_pred += ytst*0.1
    
    ifold += 1

fold 0
prepare train
prepare valid
Training until validation scores don't improve for 200 rounds.
[50]	valid_0's auc: 0.637455
[100]	valid_0's auc: 0.640221
[150]	valid_0's auc: 0.644012
[200]	valid_0's auc: 0.648524
[250]	valid_0's auc: 0.65384
[300]	valid_0's auc: 0.657828
[350]	valid_0's auc: 0.661973
[400]	valid_0's auc: 0.665733
[450]	valid_0's auc: 0.668948
[500]	valid_0's auc: 0.671543
[550]	valid_0's auc: 0.673531
[600]	valid_0's auc: 0.675258
[650]	valid_0's auc: 0.67673
[700]	valid_0's auc: 0.677881
[750]	valid_0's auc: 0.679041
[800]	valid_0's auc: 0.679671
[850]	valid_0's auc: 0.680228
[900]	valid_0's auc: 0.680742
[950]	valid_0's auc: 0.681224
[1100]	valid_0's auc: 0.682394
[1150]	valid_0's auc: 0.682544
[1200]	valid_0's auc: 0.682593
[1250]	valid_0's auc: 0.682739
[1300]	valid_0's auc: 0.682902
[1350]	valid_0's auc: 0.682769
[1400]	valid_0's auc: 0.68277
[1450]	valid_0's auc: 0.682804
[1500]	valid_0's auc: 0.68272
Early stopping, best iteration is:
[1303]	valid_0's auc: 0

In [16]:
print(scores,np.mean(scores))
roc_auc_score(X.target.values, y_oof.target.values)

[0.68292413655060324, 0.67649067087887793, 0.67748300747367651, 0.68149412930654885, 0.67871625837012206] 0.679421640516


0.67935701957848771

In [17]:
model_name = 'sparse_10folds_timp'

results_dir = './results/'
np.save(results_dir + 'train_' + model_name +'.npy', y_oof.target.values)
sample_sub = pd.read_table(data_dir+'mlboot_test.tsv')

In [18]:
sub = x_te[['uid','target']].copy()
sub['target'] = y_pred
sub.columns = ['cuid','target']
sample_sub = sample_sub.merge(sub, on='cuid', how='left')
np.save(results_dir + 'test_' + model_name +'.npy', sample_sub.target.values)
print('isnull?',sample_sub.target.isnull().any())
sample_sub.head()

isnull? False


,cuid,target
0,888b238b4d14c03173baa375a739f6bc,0.056891
1,ac4b8244f3ae82df511b002257473c11,0.030653
2,483d8b91e49522c8a5bbe37f3872c749,0.048096
3,4c7ec46a0e88a7e1e1cedd2d526d5d61,0.018669
4,fdbfba9842ff0bf86d600eb334c7c42b,0.019849


In [19]:
sample_sub[['target']].to_csv(results_dir + model_name + '.csv', header=False, index=False)